# Hyperopt and MLFlow implementation

## 1. Simple MLFlow implementation

In [10]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/05/08 11:33:53 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/gabi/codes/ml_ops/02_experiment_tracking/mlflow-examples/mlruns/2', creation_time=1746696833594, experiment_id='2', last_update_time=1746696833594, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
# read the data and treat it
# change compression if needed

def read_dataframe(filename, compression='gzip'):
    df = pd.read_csv(filename, compression=compression)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe('./data/green_tripdata_2021-01.csv.gz')
df_val = read_dataframe('./data/green_tripdata_2021-02.csv.gz')

/var/folders/55/x1yqz6851xj6j2f0smdhs4jh0000gn/T/ipykernel_39063/697616853.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, compression=compression)


In [5]:
len(df_train), len(df_val)

(73908, 61921)

In [6]:
# We will use feature engineering to create a new feature called PU_DO 
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
# set the features we will use
categorical = ['PU_DO'] 
numerical = ['trip_distance']

# We will use the DictVectorizer to convert categorical variables into a format that can be used by the model
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
# set the target variable

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
# We will use the LinearRegression model to train the data
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715210533294

In [13]:
# save the model
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
# save the model with mlflow
with mlflow.start_run():

    mlflow.set_tag("developer", "gabi")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

## 2.0 Using xgb and Hyperopt

In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
# first we need to define the objective function
def objective(params):
    # the parameters are passed both to mlflow and to xgboost
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)

        # configure the parameters of the model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

        # once the model is trained, we can save it and make predictions
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)

        mlflow.log_metric("rmse", rmse)
        
    # we return the loss, which is the value we want to minimize
    # and the status of the optimization
    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
# define the search space

# note: when we use hp.loguniform, we are exploring the range exponention(low value, high value)

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),  # here we need to convert to int, as this value only takes int
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

# we can use the fmin function to minimize the objective function
best_result = fmin(
    fn=objective, # the function we defined above
    space=search_space, # the search space we defined above
    algo=tpe.suggest, # the algorithm we will use to search the space
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/usr/local/anaconda3/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [11:53:32] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.67987                          
[1]	validation-rmse:11.19362                          
[2]	validation-rmse:10.74946                          
[3]	validation-rmse:10.34531                          
[4]	validation-rmse:9.97723                           
[5]	validation-rmse:9.64195                           
[6]	validation-rmse:9.34106                           
[7]	validation-rmse:9.06745                           
[8]	validation-rmse:8.81763                           
[9]	validation-rmse:8.59417                           
[10]	validation-rmse:8.39069                          
[11]	validation-rmse:8.20894                          
[12]	validation-rmse:8.04273                          
[13]	validation-rmse:7.89479                          
[14]	validation-rmse:7.76631                          
[15]	validation-rmse:7.64668                          
[16]	validation-rmse:7.53888                          
[17]	validation-rmse:7.44458                          
[18]	valid

/usr/local/anaconda3/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [11:54:47] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:9.08251                                                      
[1]	validation-rmse:7.61887                                                      
[2]	validation-rmse:6.98531                                                      
[3]	validation-rmse:6.71374                                                      
[4]	validation-rmse:6.58413                                                      
[5]	validation-rmse:6.51805                                                      
[6]	validation-rmse:6.48089                                                      
[7]	validation-rmse:6.46163                                                      
[8]	validation-rmse:6.44859                                                      
[9]	validation-rmse:6.43788                                                      
[10]	validation-rmse:6.43118                                                     
[11]	validation-rmse:6.42704                                                     
[12]	validation-

...

[0]	validation-rmse:11.33087                                                    
[1]	validation-rmse:10.57274                                                    
[2]	validation-rmse:9.92313                                                     
[3]	validation-rmse:9.37048                                                     
[4]	validation-rmse:8.90145                                                     
[5]	validation-rmse:8.50477                                                     
[6]	validation-rmse:8.17333                                                     
[7]	validation-rmse:7.89571                                                     
[8]	validation-rmse:7.66183                                                     
[9]	validation-rmse:7.46854                                                     
[10]	validation-rmse:7.30827                                                    
[11]	validation-rmse:7.17497                                                    
[12]	validation-rmse:7.06332

/usr/local/anaconda3/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [12:16:32] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:9.41418                                                     
[1]	validation-rmse:7.96261                                                     
[2]	validation-rmse:7.26157                                                     
[3]	validation-rmse:6.92383                                                     
[4]	validation-rmse:6.75727                                                     
[5]	validation-rmse:6.66531                                                     
[6]	validation-rmse:6.61338                                                     
[7]	validation-rmse:6.58387                                                     
[8]	validation-rmse:6.56242                                                     
[9]	validation-rmse:6.55069                                                     
[10]	validation-rmse:6.54336                                                    
[11]	validation-rmse:6.54112                                                    
[12]	validation-rmse:6.53815

/usr/local/anaconda3/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [12:16:51] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:6.77292                                                     
[1]	validation-rmse:6.67079                                                     
[2]	validation-rmse:6.64584                                                     
[3]	validation-rmse:6.63224                                                     
[4]	validation-rmse:6.62055                                                     
[5]	validation-rmse:6.61275                                                     
[6]	validation-rmse:6.60807                                                     
[7]	validation-rmse:6.58705                                                     
[8]	validation-rmse:6.58034                                                     
[9]	validation-rmse:6.57478                                                     
[10]	validation-rmse:6.57433                                                    
[11]	validation-rmse:6.57251                                                    
[12]	validation-rmse:6.56991

## 3.0 Autlog

In [ ]:
# autlog is a function that automatically logs the parameters, metrics and artifacts of the model
# it is a very useful function that can be used to log the model without having to write the code
# it only works with certain models, like xgboost, lightgbm and catboost ---> check website for full list


# you can disable autologging by using the disable parameter if you don't want to log the model
# mlflow.xgboost.autolog(disable=True) 

## 4.0 Model management

In [21]:
with mlflow.start_run():

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    # we can use the best result from the hyperopt optimization
    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    # log the parameters to mlflow
    mlflow.log_params(best_params)

    # we can use the best parameters to train the model
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    # once the model is trained, we can save it and make predictions
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # we will export the preprocessor as well
    # the preprocessor is the DictVectorizer we used to transform the data
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    # log the preprocessor - the dictvectorizer, to mlflow
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/usr/local/anaconda3/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [12:25:18] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

/usr/local/anaconda3/lib/python3.12/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [12:25:54] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/05/08 12:26:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


# 4.0 Loading a model

In [ ]:
# we pass the run id
logged_model = "runs:/0eeed647ce134080b75df6742764b655/models_mlflow"

# load model as a PyFuncModel
loaded_model = mlflow.pyfunc.load_model(logged_model)

# predict on pandas DataFrame
import pandas as pd
#loaded_model.predict(pd.DataFrame({'', 'trip_distance': [1.0]}))  # you need to pass the data in the same format as the model was trained


In [26]:
#check the loaded model
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 0eeed647ce134080b75df6742764b655

You can also load the model as a xgboost model (in this case)

In [27]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [28]:
# check the xgboost object
xgboost_model

# 5.0 Search with a lot of models

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
